https://blog.devgenius.io/sculpting-language-gpt-2-fine-tuning-with-lora-1caf3bfbc3c6

In [1]:
import json
import torch
import numpy as np
import transformers
import pandas as pd
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

d:\diskA\git-repo\gpt2-finetune\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare Dataset

In [2]:
def train_test_split(df: pd.DataFrame, test_size=0.3, random_state: int =None):
    if random_state:
        np.random.seed(random_state)
    
    # Shuffle indices
    shuffled_indices = np.random.permutation(len(df))
    
    # Split into train and test indices
    test_size = int(len(df) * test_size)
    test_indices = shuffled_indices[:test_size]
    train_indices = shuffled_indices[test_size:]
    
    # Return train and test splits
    return df.iloc[train_indices], df.iloc[test_indices]

In [3]:
# Load the dataset
dataset = load_dataset('json', data_files={'train': 'dataset/data_eng.json'})

# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Display the number of rows before removing duplicates
print(f"Number of rows before deduplication: {df.shape[0]}")

# Remove duplicates
df = df.drop_duplicates()

# Display the number of rows after removing duplicates
print(f"Number of rows after deduplication: {df.shape[0]}")

# Remove \r characters from the text in new_df
df['text'] = "Extract the job title from the provided text\ntext: " + df["input"] + "\njob title: " + df["target"] + " <STOP>"

# Split the dataset into train, validation, and test sets (70% train, 15% validation, 15% test)
train_df, val_df = train_test_split(df[['text']], test_size=0.3, random_state=42)
# val_df, test_df = train_test_split(temp_df, test_size=0.3, random_state=42)


print(f"Train Dataframe {train_df.shape[0]}")
# print(f"Test Dataframe {test_df.shape[0]}")
print(f"Validation Dataframe {val_df.shape[0]}")

# Convert DataFrames to list of dictionaries for JSON format
train_data = train_df.to_dict(orient='records')
val_data = val_df.to_dict(orient='records')
# test_data = test_df.to_dict(orient='records')

# Save the datasets to JSON files
with open('data/train_data.json', 'w') as f:
    json.dump(train_data, f)

with open('data/val_data.json', 'w') as f:
    json.dump(val_data, f)

# with open('data/test_data.json', 'w') as f:
#     json.dump(test_data, f)

print("Train, validation, and test datasets created and saved successfully in JSON format!")

Number of rows before deduplication: 2600
Number of rows after deduplication: 1848
Train Dataframe 1294
Validation Dataframe 554
Train, validation, and test datasets created and saved successfully in JSON format!


# Training

## Configuration Before Training

In [4]:
cache_dir = "models"
modelID = "openai-community/gpt2"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(modelID, cache_dir=cache_dir)

# Set padding token
tokenizer.padding_side = "right"                # Set padding side to left
tokenizer.pad_token = tokenizer.eos_token      # Using eos_token as pad_token

model = AutoModelForCausalLM.from_pretrained(modelID, device_map='auto', cache_dir=cache_dir)

model.config.pad_token_id = tokenizer.pad_token_id  # Set the pad_token_id in the model config

d:\diskA\git-repo\gpt2-finetune\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# FREEZE WEIGHTS
for param in model.parameters():
    param.requires_grad = False

In [8]:
lora_config = LoraConfig(
    r=16,                      # Rank
    lora_alpha=32,            # Alpha parameter for LoRA
    lora_dropout=0.05,         # Dropout for LoRA
    bias="none",              # Choose bias (none, all, or lora)
    task_type=TaskType.CAUSAL_LM,  # Set to Causal Language Modeling
)
model = get_peft_model(model, lora_config)

d:\diskA\git-repo\gpt2-finetune\.venv\Lib\site-packages\peft\tuners\lora\layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [9]:
# Load train, validation, and test datasets
dataset = load_dataset('json', data_files={
    'train': 'data/train_data.json',
    'validation': 'data/val_data.json',
    # 'test': 'data/test_data.json'
})

Generating train split: 1293 examples [00:00, 92376.42 examples/s]
Generating validation split: 555 examples [00:00, 50359.96 examples/s]


In [10]:
import math

# Load your training data from train_data.json
with open('data/train_data.json', 'r') as file:
    train_data = json.load(file)


# Get lengths of tokenized texts
lengths = [len(tokenizer(obj['text'])['input_ids']) for obj in train_data]

def next_power_of_2(n):
    if n < 1:
        raise ValueError("Input must be a positive integer.")

    # Calculate the power of 2 using logarithm
    power = math.ceil(math.log2(n))  # Get the smallest integer >= log2(n)
    
    # Return 2 raised to the calculated power
    return 2 ** power

print("Maximum length:", max(lengths))
max_length = next_power_of_2(max(lengths))
print("max_length:",max_length)

Maximum length: 46
max_length: 64


In [11]:
# Tokenize the dataset and create labels
def tokenize_function(examples):
    tokenized = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=max_length)
    # Create labels (shifted input for language modeling)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

In [12]:
# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 555/555 [00:00<00:00, 17342.81 examples/s]


In [13]:
# Set format for PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [15]:
print_trainable_parameters(model)

trainable params: 589824 || all params: 125029632 || trainable%: 0.4717473694555863


In [16]:
batch = 4
training_args = TrainingArguments(
    per_device_train_batch_size=batch,
    gradient_accumulation_steps=batch,
    warmup_steps=10,
    max_steps=500,
    # num_train_epochs=3, 
    learning_rate=2e-4,
    logging_steps=batch*2,
    output_dir='outputs',
    auto_find_batch_size=True
)

In [17]:
# Initialize Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],  # Use validation set here
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

max_steps is given, it will override any value given in num_train_epochs


In [18]:
model.config.use_cache = False

In [19]:
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]d:\diskA\git-repo\gpt2-finetune\.venv\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  0%|          | 1/500 [00:00<04:09,  2.00it/s]

{'loss': 5.7891, 'grad_norm': 0.99388587474823, 'learning_rate': 2e-05, 'epoch': 0.01}


  0%|          | 2/500 [00:00<02:52,  2.88it/s]

{'loss': 5.6777, 'grad_norm': 0.8763487339019775, 'learning_rate': 4e-05, 'epoch': 0.02}


  1%|          | 3/500 [00:00<02:27,  3.38it/s]

...

100%|██████████| 500/500 [02:07<00:00,  3.50it/s]

{'loss': 1.2415, 'grad_norm': 0.7012266516685486, 'learning_rate': 0.0, 'epoch': 6.17}


100%|██████████| 500/500 [02:11<00:00,  3.79it/s]

{'train_runtime': 131.9597, 'train_samples_per_second': 60.625, 'train_steps_per_second': 3.789, 'train_loss': 1.472257623553276, 'epoch': 6.17}


TrainOutput(global_step=500, training_loss=1.472257623553276, metrics={'train_runtime': 131.9597, 'train_samples_per_second': 60.625, 'train_steps_per_second': 3.789, 'total_flos': 262511987392512.0, 'train_loss': 1.472257623553276, 'epoch': 6.172839506172839})

In [20]:
torch.save(model.state_dict(), 'lora.pt')

In [21]:
xcacs

NameError: name 'xcacs' is not defined

In [24]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

cache_dir = "models"
modelID = "openai-community/gpt2"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(modelID, cache_dir=cache_dir)

# Set padding token
tokenizer.padding_side = "right"               # Set padding side to left
tokenizer.pad_token = tokenizer.eos_token      # Using eos_token as pad_token

model = AutoModelForCausalLM.from_pretrained(modelID, device_map='auto', cache_dir=cache_dir)

model.config.pad_token_id = tokenizer.pad_token_id  # Set the pad_token_id in the model config

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model = model.to(device)
model.load_state_dict(torch.load("lora.pt", map_location=device))

# Function for inference
def generate_job_title(input_text):
    # Prepare the input
    input_text = f"{input_text}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Create attention mask
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)  # 1 for real tokens, 0 for padding

    # Move tensors to the device (GPU if available)
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)

    # Generate text with the model
    generated_outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=input_ids.shape[-1] + 20,  # Max length is input length + 40
        num_return_sequences=1,
        do_sample=True,                       # Enable sampling
        temperature=0.9,                     # Control randomness
        top_k=50,                            # Top-K sampling
        top_p=0.95,                          # Top-P (nucleus) sampling
        pad_token_id=tokenizer.eos_token_id  # Padding token id
    )

    # Decode and return the generated text
    generated_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
    return generated_text

input_text = """Extract the job title from the provided text\ntext: battery state estimation engineer affirmative for women\njob title:"""
input_text = """Extract the job title from the provided text\ntext: senior devops engineer 2 jobs max in last 5 years candidates from consulting firms will be rejected\njob title:"""
input_text = """Extract the job title from the provided text\ntext: senior lighting product development engineer and product line manager\njob title:"""

# input_text = "Job Title: pharmacist at community pharmacy => Extracted Title: "
predicted_title = generate_job_title(input_text)
print(f"{predicted_title}")


Extract the job title from the provided text
text: senior lighting product development engineer and product line manager
job title: lighting product development engineer <STOP> <STOP> <STOP> <STOP>


In [ ]:
xasdas

In [26]:
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

cache_dir = "models"
modelID = "openai-community/gpt2"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(modelID, cache_dir=cache_dir)

# Set padding token
tokenizer.padding_side = "left"                # Set padding side to left
tokenizer.pad_token = tokenizer.eos_token      # Using eos_token as pad_token

model = AutoModelForCausalLM.from_pretrained(modelID, device_map='auto', cache_dir=cache_dir)

model.config.pad_token_id = tokenizer.pad_token_id  # Set the pad_token_id in the model config

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model = model.to(device)
model.load_state_dict(torch.load("lora.pt", map_location=device))

with torch.no_grad():
    batch = tokenizer("""Extract the job title from the provided text
text: senior devops engineer 2 jobs max in last 5 years candidates from consulting firms will be rejected
job title:"""
        , return_tensors='pt').to(device)
    output_tokens = model.generate(**batch, max_new_tokens=25)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




 Extract the job title from the provided text
text: senior devops engineer 2 jobs max in last 5 years candidates from consulting firms will be rejected
job title: devops engineer <STOP> <STOP> <STOP> <STOP> <STOP> <ST
